# Transforma em DataFrame e limpa

In [12]:
import pandas as pd

# Lê excel e transforma em DataFrame
vendas_df = pd.read_excel('../../../data/silver/2025_11_treated.xlsx')

In [13]:
# Cria tupla de sufixos de colunas necessários
sufixos_necessários = tuple(['Vendas', 'Anúncios', 'Publicidade'])

# Remove colunas desnecessárias
for col in vendas_df.columns:

    if not col.endswith(sufixos_necessários):
        vendas_df = vendas_df.drop(col, axis=1)

In [14]:
# Filtra apenas colunas de vendas concluídas
vendas_df = vendas_df[
    # Valor total da venda precisa ser maior que 0
    (vendas_df['Total_(BRL)_Vendas'] != 0) &
    # É preciso que haja o título do anúncio, a ausência indica que a venda não foi concluída
    (vendas_df['Título_do_anúncio_Anúncios'].notna()) &
    (vendas_df['Título_do_anúncio_Anúncios'].str.strip() != '')
]

# Reseta índice do DataFrame
vendas_df = vendas_df.reset_index(drop=True)

# Identifica categoria de produtos pelo título do anúncio

In [15]:
# Cria lista que será copiada para as outras
lista_copia = [0] * len(vendas_df)

# Cria listas por categoria que serão adicionas ao DataFrame
str_bz = lista_copia.copy()
akai_fem = lista_copia.copy()
fx = lista_copia.copy()

In [16]:
# Itera por cada linha do DataFrame
for idx, row in vendas_df.iterrows():

    # Define título do anúncio e unidades vendidas
    titulo_anuncio = row['Título_do_anúncio_Anúncios'].lower()
    unidades_vendidas = int(row['Unidades_Vendas'])

    # Caso o anúncio seja de kimono
    if 'kimono' in titulo_anuncio:

        # Se contiver akai ou fem no nome, muda o elemento correspondente ao índice na lista akai_fem pelas unidades vendidas
        if ('akai' in titulo_anuncio) or ('feminin' in titulo_anuncio):
            akai_fem[idx] = unidades_vendidas
        
        # Se não for akai/fem, será str/bz. Então, muda o elemento correspondente ao índice na lista str_bz pelas unidades vendidas
        else:
            str_bz[idx] = unidades_vendidas

        # Caso o anuncio contenha faixa, muda o elemento correspondente ao índice na lista fx pelas unidades vendidas 
        if 'faixa' in titulo_anuncio:
            fx[idx] = unidades_vendidas

    # Se o anúncio não for de kimono, será de faixa. Então...
    # else:
    ## TODO

In [17]:
# Cria novas colunas no DataFrame de vendas
vendas_df['STR/BZ'] = str_bz
vendas_df['AKAI/FEM'] = akai_fem
vendas_df['FX'] = fx

In [20]:
# Salva DataFrame
vendas_df.to_csv('../../../data/gold/2025_11_final.csv', index=False)